In [ ]:
import os

#make files if don't already exist
os.makedirs("paired_proc", exist_ok=True)
os.makedirs("paired_unproc", exist_ok=True)
os.makedirs("qza_files", exist_ok=True)
os.makedirs("seq_reads", exist_ok=True)
os.makedirs("unpaired_proc", exist_ok=True)
os.makedirs("unpaired_unproc", exist_ok=True)

In [9]:
%%bash -s "$fastq_file"

#check if files are PHRED33
check_fastq_encoding() {
    # Check if a file is provided as an argument
    if [ $# -ne 1 ]; then
        echo "Usage: $0 <fastq_file>"
        exit 1
    fi
    
    fastq_file="$1"

    # Read the first quality line from the FASTQ file
    quality_line=$(head -n 40 "$fastq_file" | grep -E '^\+[[:space:]]*$' -A 1 | tail -n 1)

    # Loop through each character in the quality line and check its ASCII value
    phred33=1
    phred64=1
    for (( i=0; i<${#quality_line}; i++ )); do
        char=$(printf "%d" "'${quality_line:$i:1}")
        if [ $char -lt 33 ] || [ $char -gt 73 ]; then
            phred33=0
        fi
        if [ $char -lt 64 ] || [ $char -gt 104 ]; then
            phred64=0
        fi
    done

    # Output the result
    if [ $phred33 -eq 1 ]; then
        echo "PHRED 33 encoding detected."
    elif [ $phred64 -eq 1 ]; then
        echo "PHRED 64 encoding detected."
    else
        echo "Unknown encoding format."
    fi
}

processed="$(pwd)/processed"

echo "$processed/ERR1351807_1.fastq.gz"

check_fastq_encoding "$processed/ERR1351807_1.fastq.gz"


/home/miloscola/augene/training_data/processed/ERR1351807_1.fastq.gz
PHRED 33 encoding detected.


In [1]:
%%bash

current_dir=$(pwd)

#gzip any randomly unzipped files
for file in $current_dir/seq_reads/*.fastq; do
    # Re-zip each file
    gzip -f "$file"
    #remove unzipped file
    #rm "$file"
done

for file in $current_dir/seq_reads/*_R1_001.fastq.gz; do
    # Extract the prefix (e.g., ERR1351807) from the filename
    prefix="${file%_R1_001.fastq.gz}"
    
    # Rename the forward read file
    mv "$file" "${prefix}_1.fastq.gz"
    
done

for file in $current_dir/seq_reads/*_R2_001.fastq.gz; do
    # Extract the prefix (e.g., ERR1351807) from the filename
    prefix="${file%_R2_001.fastq.gz}"
    
    # Rename the forward read file
    mv "$file" "${prefix}_2.fastq.gz"
    
done
                                                                                                                                    
for file in $current_dir/seq_reads/*.fastq; do
    if [[ $file == *.fastq ]]; then
        echo "$file"
    fi
done 

/home/miloscola/augene/training_data/seq_reads/*.fastq


gzip: /home/miloscola/augene/training_data/seq_reads/*.fastq: No such file or directory


In [6]:
%%bash
#LIST FILES
current_dir=$(pwd)

for file in $current_dir/seq_reads/*; do
    # Extract the prefix (e.g., ERR1351807) from the filename
    echo $file
    
done

/home/miloscola/augene/training_data/seq_reads/ERR1351820_R1_001.fastq.gz
/home/miloscola/augene/training_data/seq_reads/ERR1351820_R2_001.fastq.gz
/home/miloscola/augene/training_data/seq_reads/ERR1351821_R1_001.fastq.gz
/home/miloscola/augene/training_data/seq_reads/ERR1351821_R2_001.fastq.gz
/home/miloscola/augene/training_data/seq_reads/ERR1351822_R1_001.fastq.gz
/home/miloscola/augene/training_data/seq_reads/ERR1351822_R2_001.fastq.gz
/home/miloscola/augene/training_data/seq_reads/ERR1351823_R1_001.fastq.gz
/home/miloscola/augene/training_data/seq_reads/ERR1351823_R2_001.fastq.gz
/home/miloscola/augene/training_data/seq_reads/ERR1351824_R1_001.fastq.gz
/home/miloscola/augene/training_data/seq_reads/ERR1351824_R2_001.fastq.gz
/home/miloscola/augene/training_data/seq_reads/ERR1351825_R1_001.fastq.gz
/home/miloscola/augene/training_data/seq_reads/ERR1351825_R2_001.fastq.gz
/home/miloscola/augene/training_data/seq_reads/ERR1351826_R1_001.fastq.gz
/home/miloscola/augene/training_data/s

In [3]:
%%bash
#NO FOR LOOP TEST
#Sort Raw Data

# Enable extended pattern matching
shopt -s extglob

current_dir=$(pwd)
seq_reads="seq_reads"
seq_reads="$current_dir/$seq_reads"


#TEST WITHOUT FOR LOOP
files=($(ls -1 "$seq_reads" | head -n 2 | sort))
    
#echo "Moved ${files[0]} and ${files[1]} to processing directory."

if [[ "${files[0]}" == *_1.fastq.gz && "${files[1]}" == *_2.fastq.gz ]]; then
    # Move both files to the processing directory
    filename=${files[0]}
    sample_id="${filename%_1.fastq.gz}"
    mkdir "$current_dir/paired_unproc/$sample_id" 
    mv "$seq_reads/${files[0]}" "$seq_reads/${files[1]}" "$current_dir/paired_unproc/$sample_id"

else
    # Move only the top file to the processing directory
    filename=${files[0]}
    sample_id="${filename%_1.fastq.gz}"
    mkdir "$current_dir/unpaired_unproc/$sample_id" 
    mv "$seq_reads/${files[0]}" "$current_dir/unpaired_unproc/$sample_id"
fi

In [3]:
%%bash
#Sort Raw Data #REPLACE WITH CODE ABOVE WHEN DONE TESTING

# Enable extended pattern matching
shopt -s extglob

current_dir=$(pwd)
seq_reads="seq_reads"
seq_reads="$current_dir/$seq_reads"

# Enter the loop
while [ "$(ls -A $seq_reads)" ]; do
    files=($(ls -1 "$seq_reads" | head -n 2 | sort))
    
    #echo "Moved ${files[0]} and ${files[1]} to processing directory."

    if [[ "${files[0]}" == *_1.fastq.gz && "${files[1]}" == *_2.fastq.gz ]]; then
        # Move both files to the processing directory
        filename=${files[0]}
        sample_id="${filename%_1.fastq.gz}"
        mkdir "$seq_reads/$sample_id" 
        mv "$seq_reads/${files[0]}" "$seq_reads/${files[1]}" "$current_dir/paired_unproc/$sample_id"

    else
        # Move only the top file to the processing directory
        mv "$seq_reads/${files[0]}" "$current_dir/unpaired_unproc/"
    fi
done

Process is interrupted.


In [5]:
%%bash
#Remove old manifest and Generate new Manifests
shopt -s extglob

current_dir=$(pwd)
paired_unproc="$current_dir/paired_unproc"
unpaired_unproc="$current_dir/unpaired_unproc"

# Enable extended pattern matching
rm "${paired_unproc}/manifest.tsv"
rm "${unpaired_unproc}/manifest.tsv"

#generate manifest in paired folder

# Get the list of folder names in the specified directory and store in an array
samples=($(find "$paired_unproc" -maxdepth 1 -type d -exec basename {} \;))

#generate new manifest
cp "${current_dir}/DataFrame_2_Pruned_paired.tsv" "${paired_unproc}/manifest.tsv"

for sample in "${samples[@]}"; do
    sample_path="${paired_unproc}/${sample}"
    reads=($(ls -1 "$sample_path"))
    
    awk -v row="$row_index" -v col="$column_index" -v val="$new_value" -F'\t' 'BEGIN {OFS=FS} { if (NR == row) $col = val; print }' "$tsv_file" > modified.tsv
done

#generate manifest in unpaired folder



paired_unproc
ERR1351807


rm: cannot remove '/home/miloscola/augene/training_data/paired_unproc/manifest.tsv': No such file or directory
rm: cannot remove '/home/miloscola/augene/training_data/unpaired_unproc/manifest.tsv': No such file or directory


In [6]:
import os
import shutil
import pandas as pd

# Define paths
current_dir = os.getcwd()
paired_unproc = os.path.join(current_dir, 'paired_unproc')
unpaired_unproc = os.path.join(current_dir, 'unpaired_unproc')
paired_master_manifest = os.path.join(current_dir, 'DataFrame_2_Pruned_paired.tsv')
unpaired_master_manifest = os.path.join(current_dir, 'DataFrame_2_Pruned_paired.tsv')

# Remove old manifest files if they exist
paired_manifest = os.path.join(paired_unproc, 'manifest.tsv')
unpaired_manifest = os.path.join(unpaired_unproc, 'manifest.tsv')

if os.path.exists(paired_manifest):
    os.remove(paired_manifest)

if os.path.exists(unpaired_manifest):
    os.remove(unpaired_manifest)

### Generate manifest in paired folder ####
samples = [f for f in os.listdir(paired_unproc) if os.path.isdir(os.path.join(paired_unproc, f))]

# Read the manifest file
df = pd.read_csv(paired_master_manifest, sep='\t', encoding='latin1')
df.rename(columns={'Run': 'id'}, inplace=True)


# Update the paired manifest
for sample in samples:
    sample_path = os.path.join(paired_unproc, sample)
    reads = os.listdir(sample_path)
    
    print(sample)
    
    # Assuming you want to update the path in a specific column, e.g., 'file-path'
    df.loc[df['id'] == sample, 'forward-absolute-filepath'] = os.path.join(sample_path, reads[0])
    df.loc[df['id'] == sample, 'reverse-absolute-filepath'] = os.path.join(sample_path, reads[1])

print(df.head)

#remove all rows of df without sample
df = df[df['forward-absolute-filepath'] != 'na']

print(df.head)

# Save the modified TSV
df.to_csv(paired_manifest, sep='\t', index=False)
    
# You can add similar code to handle the unpaired manifest if needed
# For example:
# shutil.copy(os.path.join(current_dir, 'DataFrame_2_Pruned_unpaired.tsv'), unpaired_manifest)
# for sample in unpaired_samples:
#     sample_path = os.path.join(unpaired_unproc, sample)
#     update_manifest(unpaired_manifest, sample_path, sample)

ERR1351807
<bound method NDFrame.head of               id  Year_of_release     Release_Date       Load_Date Submission  \
0     ERR1351807             2017  8/21/2017 23:46  8/23/2017 3:56  ERA600241   
1     ERR1351808             2017  8/21/2017 23:46  8/23/2017 3:57  ERA600241   
2     ERR1351809             2017  8/21/2017 23:46  8/23/2017 3:57  ERA600241   
3     ERR1351810             2017  8/21/2017 23:46  8/23/2017 3:57  ERA600241   
4     ERR1351811             2017  8/21/2017 23:46  8/23/2017 3:57  ERA600241   
...          ...              ...              ...             ...        ...   
5941  SRR9660388             2020    8/8/2020 0:00  7/9/2019 14:28  SRA919014   
5942  SRR9660389             2020    8/8/2020 0:00  7/9/2019 14:28  SRA919014   
5943  SRR9660390             2020    8/8/2020 0:00  7/9/2019 14:28  SRA919014   
5944  SRR9660391             2020    8/8/2020 0:00  7/9/2019 14:29  SRA919014   
5945  SRR9660392             2020    8/8/2020 0:00  7/9/2019 14:29  

In [7]:
%%bash
#run qiime2

current_dir=$(pwd)
paired_unproc="$current_dir/paired_unproc"
paired_proc="$current_dir/paired_proc"

qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path "$paired_unproc/manifest.tsv" \
    --output-path "$current_dir/qza_files/paired-end-demux.qza" \
    --input-format PairedEndFastqManifestPhred33V2 

mv $paired_unproc/manifest.tsv $current_dir/qza_files/sample-metadata.tsv

#mv $paired_unproc/* $paired_proc
        
echo "initial processing complete"
        
qiime dada2 denoise-paired \
    --i-demultiplexed-seqs qza_files/paired-end-demux.qza \
    --p-trim-left-f 13 \
    --p-trim-left-r 13 \
    --p-trunc-len-f 150 \
    --p-trunc-len-r 150 \
    --o-representative-sequences "$current_dir/qza_files/rep-seqs-dada2.qza" \
    --o-table "$current_dir/qza_files/table-dada2.qza" \
    --o-denoising-stats "$current_dir/qza_files/stats-dada2.qza"
        
echo "denoising complete"
        
qiime feature-table summarize \
    --i-table "$current_dir/qza_files/table-dada2.qza" \
    --o-visualization "$current_dir/qza_files/table-dada2.qzv" \
    --m-sample-metadata-file "$current_dir/qza_files/sample-metadata.tsv"
        
echo "feature table complete"
        
qiime taxa barplot \
    --i-table "$current_dir/qza_files/table-dada2.qza" \
    --i-taxonomy "$current_dir/qza_files/taxonomy.qza" \
    --m-metadata-file "$current_dir/qza_files/sample-metadata.tsv" \
    --o-visualization "$current_dir/qza_files/taxa-bar-plots.qzv"
        
echo "bar plot complete"

rm $current_dir/qza_files/sample-metadata.tsv

Imported /home/miloscola/augene/training_data/paired_unproc/manifest.tsv as PairedEndFastqManifestPhred33V2 to /home/miloscola/augene/training_data/qza_files/paired-end-demux.qza
initial processing complete
Saved FeatureTable[Frequency] to: /home/miloscola/augene/training_data/qza_files/table-dada2.qza
Saved FeatureData[Sequence] to: /home/miloscola/augene/training_data/qza_files/rep-seqs-dada2.qza
Saved SampleData[DADA2Stats] to: /home/miloscola/augene/training_data/qza_files/stats-dada2.qza
denoising complete
Saved Visualization to: /home/miloscola/augene/training_data/qza_files/table-dada2.qzv
feature table complete
bar plot complete


Usage: qiime taxa barplot [OPTIONS]

  This visualizer produces an interactive barplot visualization of
  taxonomies. Interactive features include multi-level sorting, plot
  recoloring, sample relabeling, and SVG figure export.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                         Feature table to visualize at various taxonomic
                         levels.                                    [required]
  --i-taxonomy ARTIFACT FeatureData[Taxonomy]
                         Taxonomic annotations for features in the provided
                         feature table. All features in the feature table must
                         have a corresponding taxonomic annotation. Taxonomic
                         annotations that are not present in the feature table
                         will be ignored.                           [required]
Parameters:
  --m-metadata-file METADATA...
    (multiple            The sample metadata.
     arguments will be   
     merged)

In [2]:
%%bash
#import qiime tools
qiime tools import --show-importable-formats

#qiime tools import --show-importable-types

AlignedDNAFASTAFormat
AlignedDNASequencesDirectoryFormat
AlphaDiversityDirectoryFormat
AlphaDiversityFormat
BIOMV100DirFmt
BIOMV100Format
BIOMV210DirFmt
BIOMV210Format
BooleanSeriesDirectoryFormat
BooleanSeriesFormat
Bowtie2IndexDirFmt
CasavaOneEightLanelessPerSampleDirFmt
CasavaOneEightSingleLanePerSampleDirFmt
DADA2StatsDirFmt
DADA2StatsFormat
DNAFASTAFormat
DNASequencesDirectoryFormat
DeblurStatsDirFmt
DeblurStatsFmt
DifferentialDirectoryFormat
DifferentialFormat
DistanceMatrixDirectoryFormat
EMPPairedEndCasavaDirFmt
EMPPairedEndDirFmt
EMPSingleEndCasavaDirFmt
EMPSingleEndDirFmt
ErrorCorrectionDetailsDirFmt
FastqGzFormat
FirstDifferencesDirectoryFormat
FirstDifferencesFormat
HeaderlessTSVTaxonomyDirectoryFormat
HeaderlessTSVTaxonomyFormat
ImportanceDirectoryFormat
ImportanceFormat
LSMatFormat
MultiplexedPairedEndBarcodeInSequenceDirFmt
MultiplexedSingleEndBarcodeInSequenceDirFmt
NewickDirectoryFormat
NewickFormat
OrdinationDirectoryFormat
OrdinationFormat
PairedDNASequencesDirectory

In [ ]:
%%bash
#single for loop run test

# Enable extended pattern matching
shopt -s extglob

current_dir=$(pwd)
seq_reads="seq_reads"
seq_reads="$current_dir/$seq_reads"

files=($(ls -1 "$seq_reads" | head -n 2 | sort))
    
# Move both files to the processing directory
mv "$seq_reads/${files[0]}" "$seq_reads/${files[1]}" "$current_dir/processing/Sample1/"
echo "Moved ${files[0]} and ${files[1]} to processing directory."

if [[ "${files[0]}" == *_1.fastq.gz && "${files[1]}" == *_2.fastq.gz ]]; then
    # Move both files to the processing directory
    #mv "$seq_reads/${files[0]}" "$seq_reads/${files[1]}" "$current_dir/processing/"
        
    #echo "Moved ${files[0]} and ${files[1]} to processing directory."
        
    echo "if condition triggered"
        
    #for gzip_file in $current_dir/processing/*.gz; do
    #    # Unzip each file
    #    gunzip -f "$gzip_file"
    #done
    
#    T=$(printf '\t')
    
#    cat > $current_dir/processing/manifest/manifest.tsv << EOF
#sample-id $T forward-absolute-filepath $T reverse-absolute-filepath
#sample-1 $T $current_dir/processing/Sample1/${files[0]} $T $current_dir/processing/Sample1/${files[1]}
#EOF
    
#    cat <<EOF > "$current_dir/processing/manifest/manifest.tsv"
#sample-id"\t"forward-absolute-filepath"\t"reverse-absolute-filepath
#sample-1"\t""$current_dir/processing/Sample1/${files[0]}""\t""$current_dir/processing/Sample1/${files[1]}"
#EOF
        
    qiime tools import \
        --type 'SampleData[PairedEndSequencesWithQuality]' \
        --input-path "$current_dir/processing/manifest/manifest.tsv" \
        --output-path "$current_dir/qza_files/paired-end-demux.qza" \
        --input-format PairedEndFastqManifestPhred33V2 
        
    #rm $current_dir/processing/manifest/manifest.tsv
          
    #re-zip files
    #for file in $current_dir/processing/*; do
    #   # Re-zip each file
    #    gzip -f "$file"
    #done
        
    mv $current_dir/processing/Sample1/* $current_dir/processed/
        
    echo "initial processing complete"
        
    qiime dada2 denoise-paired \
      --i-demultiplexed-seqs qza_files/paired-end-demux.qza \
      --p-trim-left-f 13 \
      --p-trim-left-r 13 \
      --p-trunc-len-f 150 \
      --p-trunc-len-r 150 \
      --o-representative-sequences "$current_dir/qza_files/rep-seqs-dada2.qza" \
      --o-table "$current_dir/qza_files/table-dada2.qza" \
      --o-denoising-stats "$current_dir/qza_files/stats-dada2.qza"
        
    echo "denoising complete"
        
    qiime feature-table summarize \
      --i-table "$current_dir/qza_files/table-dada2.qza" \
      --o-visualization "$current_dir/qza_files/table-dada2.qzv" \
      --m-sample-metadata-file "$current_dir/qza_files/sample-metadata.tsv"
        
    echo "feature table complete"
    
    #FIX COMMAND INPUTS
    qiime feature-classifier classify-sklearn \
    --i-classifier "$current_dir/classifiers/gg-13-8-99-515-806-nb-classifier.qza" \
    --i-reads "$current_dir/qza_files/rep-seqs-dada2.qza" \
    --o-classification "$current_dir/qza_files/taxonomy.qza"
    
    #FIX COMMAND INPUTS
    #qiime metadata tabulate \
    #--m-input-file "$current_dir/qza_files/taxonomy.qza" \
    #--o-visualization "$current_dir/qza_files/taxonomy.qzv"

        
    #qiime taxa barplot \
    #  --i-table "$current_dir/qza_files/table-dada2.qza" \
    #  --i-taxonomy "$current_dir/qza_files/taxonomy.qza" \
    #  --m-metadata-file "$current_dir/qza_files/sample-metadata.tsv" \
    #  --o-visualization "$current_dir/qza_files/taxa-bar-plots.qzv"
        
    #echo "bar plot complete"
        
else
    # Move only the top file to the processing directory
    echo "else condition triggered"
    break 
    #TODO: add single read seq code
    #mv "${files[0]}" processing/
    #echo "Moved ${files[0]} to processing directory."
fi

In [1]:
%%bash

# Enable extended pattern matching
shopt -s extglob

current_dir=$(pwd)
seq_reads="seq_reads"
seq_reads="$current_dir/$seq_reads"

# Enter the loop
while [ "$(ls -A $seq_reads)" ]; do
    files=($(ls -1 "$seq_reads" | head -n 2 | sort))
    
    # Move both files to the processing directory
    mv "$seq_reads/${files[0]}" "$seq_reads/${files[1]}" "$current_dir/processing/"
    echo "Moved ${files[0]} and ${files[1]} to processing directory."

    if [[ "${files[0]}" == *_1.fastq.gz && "${files[1]}" == *_2.fastq.gz ]]; then
        # Move both files to the processing directory
        #mv "$seq_reads/${files[0]}" "$seq_reads/${files[1]}" "$current_dir/processing/"
        
        #echo "Moved ${files[0]} and ${files[1]} to processing directory."
        
        echo "if condition triggered"
        
        for gzip_file in $current_dir/processing/*.gz; do
            # Unzip each file
            gunzip -f "$gzip_file"
            # Remove the original gzip file
            rm "$gzip_file"
        done
        
        qiime tools import \
          --type 'SampleData[PairedEndSequencesWithQuality]' \
          --input-path "$current_dir/processing/" \
          --output-path "$current_dir/qza_files/paired-end-demux.qza" \
          --input-format CasavaOneEightSingleLanePerSampleDirFmt
          
        #re-zip files
        for file in $current_dir/processing/*; do
            # Re-zip each file
            gzip -f "$file"
            #remove unzipped file
            rm "$file"
        done
        
        mv "$current_dir/processing/*" "$current_dir/processed/"
        
        echo "initial processing complete"
        
        qiime dada2 denoise-paired \
          --i-demultiplexed-seqs qza_files/paired-end-demux.qza \
          --p-trim-left-f 13 \
          --p-trim-left-r 13 \
          --p-trunc-len-f 150 \
          --p-trunc-len-r 150 \
          --o-representative-sequences "$current_dir/qza_files/rep-seqs-dada2.qza" \
          --o-table "$current_dir/qza_files/table-dada2.qza" \
          --o-denoising-stats "$current_dir/qza_files/stats-dada2.qza"
        
        echo "denoising complete"
        
        qiime feature-table summarize \
          --i-table "$current_dir/qza_files/table-dada2.qza" \
          --o-visualization "$current_dir/qza_files/table-dada2.qzv" \
          --m-sample-metadata-file "$current_dir/qza_files/sample-metadata.tsv"
        
        echo "feature table complete"
        
        qiime taxa barplot \
          --i-table "$current_dir/qza_files/table-dada2.qza" \
          --i-taxonomy "$current_dir/qza_files/taxonomy.qza" \
          --m-metadata-file "$current_dir/qza_files/sample-metadata.tsv" \
          --o-visualization "$current_dir/qza_files/taxa-bar-plots.qzv"
        
        echo "bar plot complete"
        
    else
        # Move only the top file to the processing directory
        echo "else condition triggered"
        break 
        #TODO: add single read seq code
        #mv "${files[0]}" processing/
        #echo "Moved ${files[0]} to processing directory."
    fi
done

Process is interrupted.


In [1]:
%%bash

qiime info

System versions
Python version: 3.6.10
QIIME 2 release: 2020.8
QIIME 2 version: 2020.8.0
q2cli version: 2020.8.0

Installed plugins
alignment: 2020.8.0
composition: 2020.8.0
cutadapt: 2020.8.0
dada2: 2020.8.0
deblur: 2020.8.0
demux: 2020.8.0
diversity: 2020.8.0
diversity-lib: 2020.8.0
emperor: 2020.8.0
feature-classifier: 2020.8.0
feature-table: 2020.8.0
fragment-insertion: 2020.8.0
gneiss: 2020.8.0
longitudinal: 2020.8.0
metadata: 2020.8.0
phylogeny: 2020.8.0
quality-control: 2020.8.0
quality-filter: 2020.8.0
sample-classifier: 2020.8.0
taxa: 2020.8.0
types: 2020.8.0
vsearch: 2020.8.0

Application config directory
/home/miloscola/anaconda3/envs/qiime2/var/q2cli

Getting help
To get help with QIIME 2, visit https://qiime2.org


In [10]:
%%bash

# Enable extended pattern matching
shopt -s extglob

current_dir=$(pwd)
seq_reads="seq_reads"
seq_reads="$current_dir/$seq_reads"

files=($(ls -1 "$seq_reads" | head -n 2 | sort))
    
# Move both files to the processing directory
mv "$seq_reads/${files[0]}" "$seq_reads/${files[1]}" "$current_dir/processing/Sample1/"
echo "Moved ${files[0]} and ${files[1]} to processing directory."

if [[ "${files[0]}" == *_1.fastq.gz && "${files[1]}" == *_2.fastq.gz ]]; then
    # Move both files to the processing directory
    #mv "$seq_reads/${files[0]}" "$seq_reads/${files[1]}" "$current_dir/processing/"
        
    #echo "Moved ${files[0]} and ${files[1]} to processing directory."
        
    echo "if condition triggered"
        
    #for gzip_file in $current_dir/processing/*.gz; do
    #    # Unzip each file
    #    gunzip -f "$gzip_file"
    #done
    
    T=$(printf '\t')
    
    cat > $current_dir/processing/manifest/manifest.tsv << EOF
sample-id $T forward-absolute-filepath $T reverse-absolute-filepath
sample-1 $T $current_dir/processing/Sample1/${files[0]} $T $current_dir/processing/Sample1/${files[1]}
EOF
    
#    cat <<EOF > "$current_dir/processing/manifest/manifest.tsv"
#sample-id"\t"forward-absolute-filepath"\t"reverse-absolute-filepath
#sample-1"\t""$current_dir/processing/Sample1/${files[0]}""\t""$current_dir/processing/Sample1/${files[1]}"
#EOF
        
    qiime tools import \
        --type 'SampleData[PairedEndSequencesWithQuality]' \
        --input-path "$current_dir/processing/manifest/manifest.tsv" \
        --output-path "$current_dir/qza_files/paired-end-demux.qza" \
        --input-format PairedEndFastqManifestPhred33V2 
        
    #rm $current_dir/processing/manifest/manifest.tsv
          
    #re-zip files
    #for file in $current_dir/processing/*; do
    #   # Re-zip each file
    #    gzip -f "$file"
    #done
        
    mv $current_dir/processing/Sample1/* $current_dir/processed/
        
    echo "initial processing complete"
        
    qiime dada2 denoise-paired \
      --i-demultiplexed-seqs qza_files/paired-end-demux.qza \
      --p-trim-left-f 13 \
      --p-trim-left-r 13 \
      --p-trunc-len-f 150 \
      --p-trunc-len-r 150 \
      --o-representative-sequences "$current_dir/qza_files/rep-seqs-dada2.qza" \
      --o-table "$current_dir/qza_files/table-dada2.qza" \
      --o-denoising-stats "$current_dir/qza_files/stats-dada2.qza"
        
    echo "denoising complete"
        
    qiime feature-table summarize \
      --i-table "$current_dir/qza_files/table-dada2.qza" \
      --o-visualization "$current_dir/qza_files/table-dada2.qzv" \
      --m-sample-metadata-file "$current_dir/qza_files/sample-metadata.tsv"
        
    echo "feature table complete"
        
    qiime taxa barplot \
      --i-table "$current_dir/qza_files/table-dada2.qza" \
      --i-taxonomy "$current_dir/qza_files/taxonomy.qza" \
      --m-metadata-file "$current_dir/qza_files/sample-metadata.tsv" \
      --o-visualization "$current_dir/qza_files/taxa-bar-plots.qzv"
        
    echo "bar plot complete"
        
else
    # Move only the top file to the processing directory
    echo "else condition triggered"
    break 
    #TODO: add single read seq code
    #mv "${files[0]}" processing/
    #echo "Moved ${files[0]} to processing directory."
fi

Moved ERR1351843_R1_001.fastq.gz and ERR1351843_R2_001.fastq.gz to processing directory.
if condition triggered
Imported /home/miloscola/augene/training_data/processing/manifest/manifest.tsv as PairedEndFastqManifestPhred33V2 to /home/miloscola/augene/training_data/qza_files/paired-end-demux.qza
initial processing complete
denoising complete
feature table complete
bar plot complete


Plugin error from dada2:

  An error was encountered while running DADA2 in R (return code 1), please inspect stdout and stderr to learn more.

Debug info has been saved to /tmp/qiime2-q2cli-err-zwl7r5vc.log
There was an issue with loading the file /home/miloscola/augene/training_data/qza_files/sample-metadata.tsv as metadata:

  Metadata file path doesn't exist, or the path points to something other than a file. Please check that the path exists, has read permissions, and points to a regular file (not a directory): /home/miloscola/augene/training_data/qza_files/sample-metadata.tsv

  There may be more errors present in the metadata file. To get a full report, sample/feature metadata files can be validated with Keemei: https://keemei.qiime2.org

  Find details on QIIME 2 metadata requirements here: https://docs.qiime2.org/2020.8/tutorials/metadata/

There was an issue with loading the file /home/miloscola/augene/training_data/qza_files/sample-metadata.tsv as metadata:

  Metadata file 

In [ ]:
#TODO:
#0) standardize file names -DONE
#1) make seprate folders for each sample **account for single and paired reads
#2) make a manifest with sample names corresponding 
#   to the sample names in the metadata TSV for all files -DONE
#3) run pipeline with all files all at once